<a href="https://colab.research.google.com/github/Zuhair0000/tensorflow_bootcamp/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [2]:
!nvidia-smi

Wed Feb 11 06:01:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P8             18W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

with zipfile.ZipFile('10_food_classes_10_percent.zip', 'r') as zip_ref:
  zip_ref.extractall('.')
  print("Unzipping complete")

--2026-02-11 06:01:25--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.24.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  22.5MB/s    in 8.1s    

2026-02-11 06:01:34 (19.8 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]

Unzipping complete


In [4]:
import os

for dirpath, dirnames, filenames in os.walk("/content/10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'. ")

There are 2 directories and 0 images in '/content/10_food_classes_10_percent'. 
There are 10 directories and 0 images in '/content/10_food_classes_10_percent/train'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ice_cream'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/fried_rice'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/steak'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/pizza'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_wings'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_curry'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ramen'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/hamburger'. 
There are 0 directories and 75 images in '/content/10_food_c

In [5]:
train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

In [6]:
import tensorflow as tf

train_data = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                         image_size=(224, 224),
                                                         batch_size=32,
                                                         seed=42)

test_data = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                         image_size=(224, 224),
                                                         batch_size=32,
                                                         seed=42)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


# Callbacks

### Tensorboard callback

This is the most powerful visualization tool in TensorFlow. Instead of staring at text output in your terminal, TensorBoard creates an interactive website running on your local machine where you can see graphs of your loss and accuracy live.

Why use it?

* Compare different experiments (e.g., "Model A with 3 layers" vs "Model B with 5 layers").

* Check if your model is overfitting (by comparing train vs validation lines).

* Visualize the model architecture graph.

In [7]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback

# Craeting models using Tensorflow Hub

in the past we have used tensorflow to buidl our models layer by layer from scratch
now we are going to do a similar process, but the majority of the model's layers are going to come from tensorflow hub


In [8]:
resnet_url = 'https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/feature-vector/1'

efficientnet_url = 'https://www.kaggle.com/models/tensorflow/efficientnet/TensorFlow2/b0-feature-vector/1'

In [9]:
import tensorflow as tf
import tensorflow_hub as hub

In [14]:
def create_model(model_url, num_classes=10):

  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name='feature_extractor_layer')

  model = tf.keras.Sequential([
      tf.keras.layers.Input(shape=(224, 224, 3)),

      tf.keras.layers.Rescaling(1/255.),

      feature_extractor_layer,

      tf.keras.layers.Dense(num_classes, activation='softmax', name="output_layer")
  ])

  return model

In [15]:
resnet_model = create_model(resnet_url,
                            num_classes=10)

In [16]:
resnet_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='Adam',
                     metrics=['accuracy'])

In [19]:
resnet_model.fit(train_data,
                 epochs=5,
                 steps_per_epoch = len(train_data),
                 validation_data=test_data,
                 validation_steps=len(test_data),
                 callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub', experiment_name='resnet50v2')])

Saving TensorBoard log files to : tensorflow_hub/resnet50v2/20260211-060825
Epoch 1/5
24/24 [==============================] - 13s 512ms/step - loss: 0.5535 - accuracy: 0.8600 - val_loss: 0.5897 - val_accuracy: 0.8220
Epoch 2/5
24/24 [==============================] - 12s 504ms/step - loss: 0.4177 - accuracy: 0.9053 - val_loss: 0.5788 - val_accuracy: 0.8132
Epoch 3/5
24/24 [==============================] - 12s 500ms/step - loss: 0.3331 - accuracy: 0.9307 - val_loss: 0.5511 - val_accuracy: 0.8240
Epoch 4/5
24/24 [==============================] - 12s 505ms/step - loss: 0.2743 - accuracy: 0.9507 - val_loss: 0.5159 - val_accuracy: 0.8384
Epoch 5/5
24/24 [==============================] - 12s 514ms/step - loss: 0.2216 - accuracy: 0.9707 - val_loss: 0.5043 - val_accuracy: 0.8376


In [20]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 feature_extractor_layer (K  (None, 2048)              23561152  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23581642 (89.96 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________
